# Visualisation

## Principal component analysis

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns

#Import data
train = np.load("fashion_train.npy")
test = np.load("fashion_test.npy")

#Split data in features and classes
X_train = train[:,:-1]
Y_train = train[:,-1]
X_test = test[:,:-1]
Y_test = test[:,-1]


[0 1 0 ... 3 0 1]


## Linear Discriminant Analysis

# Classification

## Naive Bayes Classifier

## Second classifier

## Third classifier